In [2]:
import tweepy

from tweepy import OAuthHandler
from tweepy import API
import json

import time
from tweepy.streaming import StreamListener
from tweepy import Stream

import pandas as pd


C_KEY = ''
C_SECRET = ''
A_TOKEN_KEY = ''
A_TOKEN_SECRET = ''

auth = tweepy.OAuthHandler(C_KEY, C_SECRET)
auth.set_access_token(A_TOKEN_KEY, A_TOKEN_SECRET)
api = tweepy.API(auth)

In [3]:
keywords = ['CNN', 'FoxNews', 'MSNBC', 'NPR', 'cspan']

In [9]:
# collect search data

filename = 'search_2020-03-10.json' # update this each time
results = []
for keyword in keywords:
    keyword_filename = keyword +"/" + keyword + "_" + filename
    outFile = open(keyword_filename, 'w')
    
    posts = api.search(q=keyword,count=1000, result_type='mixed', until='2020-03-11') # update 'until' each time
    
    count = 0
    for tweet in posts:
        outFile.write(json.dumps(tweet._json))
        outFile.write('\n')
        count += 1
    results.append((keyword, count))
        
    outFile.close()
print(results)

[('CNN', 100), ('FoxNews', 100), ('MSNBC', 100), ('NPR', 100), ('cspan', 100)]


In [ ]:
# check that data was collected correctly

with open('CNN/CNN' + "_" + filename) as f:
    tweets = [json.loads(line) for line in f]
    data = pd.DataFrame(columns=['user','id', 'date']);
    for twt in tweets:
        data = data.append({'id': twt['id'], 'date': twt['created_at'], 'user': str(twt['user']['screen_name'])}, ignore_index = True)

data

In [ ]:
# class for collecting streaming data

class MyListener(StreamListener):

    def __init__(self, filename, time_limit=10):
        self.start_time = time.time()
        self.limit = time_limit
        self.outFile = open(filename, 'w')
        super(MyListener, self).__init__()
    
    def on_data(self, data):
        if (time.time() - self.start_time) < self.limit:
            self.outFile.write(data.strip())     
            self.outFile.write('\n') 
            return True
        else:
            self.outFile.close()
            return False

    def on_error(self, status):
        print(status)

In [ ]:
# collect streaming data

filename = "streaming_2020-03-03-2230.json"
duration = 60 * 20
myStream = Stream(auth, MyListener(filename, time_limit=duration))
myStream.filter(track=keywords)

In [ ]:
# check that streaming data was collected correctly

with open(filename) as f:
    tweets = [json.loads(line) for line in f]
    data = pd.DataFrame(columns=['user','text', 'date']);
    for twt in tweets:
        try:
            data = data.append({'text': twt['text'], 'date': twt['created_at'], 'user': str(twt['user']['screen_name'])}, ignore_index = True)
        except KeyError:
            KeyError